<a href="https://colab.research.google.com/github/thesalmonification/DSCI400_Revamp/blob/master/PhaseVocoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
############################################################
"""This code was designed to implement phase vocoding, a technique used
to shorten signals in time without destroying their frequency content. This
function is vital to our project as unaltered EEG signals varied in length for 
each session. Phase vocoding was implemented using the popular audio editing 
library librosa"""


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import librosa
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#Import the waveform of the HDF5
keys = pd.read_csv('/content/drive/Shared drives/DSCI400/Waveform_Data_Keys.csv',header=None)
keys = list(keys[0])

In [0]:
#iterate over all keys to grab all waveforms...

#I need to shorten all longer waveforms to be equal to the shortest of all sessions
#Therefore, I need to know the length of the shortest waveform. Let's find it...
shortest_length = float('inf')

for key in keys:
  #print(key)
  session_df = pd.read_hdf('/content/drive/Shared drives/DSCI400/Waveform_Data.h5',key=key)
  session_length = session_df.shape[0]
  if session_length < shortest_length:
    shortest_length = session_length
print("Shortest Length is: " + str(shortest_length))


In [0]:
#Now for every key, phase vocode every channel and save to a new HDF5 file...

for key in keys:
  #print(key)
  session_df = pd.read_hdf('/content/drive/Shared drives/DSCI400/Waveform_Data.h5',key=key)
  session_length = session_df.shape[0]
  
  #Here I need to calculate the phase vocoding fraction to shorten any longer waveforms...
  fraction = session_length / shortest_length


  #Build the first vocoded dataframe so I may append to it
  waveform = session_df[session_df.columns[0]].to_numpy()
  STFT = librosa.stft(waveform,256)
  STFT_fast = librosa.phase_vocoder(STFT, fraction)
  waveform_fast = librosa.istft(STFT_fast)

  waveform_array = waveform_fast
  waveform_array = waveform_array.reshape(-1,1).T


  #Iterate over all others chs to build the shortened waveform array
  for ch in session_df.columns[1:]:
    waveform = session_df[ch].to_numpy()


    #Here I do vocoding thru a STFT and the librosa phase vocoder function...
    STFT = librosa.stft(waveform,256)
    STFT_fast = librosa.phase_vocoder(STFT, fraction)
    waveform_fast = librosa.istft(STFT_fast)
    waveform_fast = waveform_fast.reshape(-1,1).T

    #Append to the growing waveform array
    waveform_array = np.concatenate((waveform_array,waveform_fast),axis=0)

  #Save waveform to New HDF5 file...
  waveform_df = pd.DataFrame(waveform_array,index=session_df.columns).T

  waveform_df.to_hdf('/content/drive/Shared drives/DSCI400/Waveform_Vocoded_Unequal_Data.h5',key=key,complevel=4,format='table')


     

In [0]:
#Since the vocoder isn't perfect, reopen the file and shorten all to the newest resulting shortest value...


shortest_length = float('inf')

for key in keys:
  #print(key)
  session_df = pd.read_hdf('/content/drive/Shared drives/DSCI400/Waveform_Vocoded_Unequal_Data.h5',key=key)
  session_length = session_df.shape[0]
  if session_length < shortest_length:
    shortest_length = session_length

print("Shortest Length is: " + str(shortest_length))

In [0]:
#Save to a final file that uses the shortest length of all files...

for key in keys:
  session_df = pd.read_hdf('/content/drive/Shared drives/DSCI400/Waveform_Vocoded_Unequal_Data.h5',key=key)
  
  session_df.to_hdf('/content/drive/Shared drives/DSCI400/Waveform_Vocoded_Equalized_Data.h5',key=key,complevel=4,format='table')
